# Visualisation 

In [1]:
import numpy as np
import os
import cv2 as cv2
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# Utilities

In [2]:
def show(img, factor=1,name="image"):
    """ 
    show an image until the escape key is pressed
    :param factor: scale factor (default 1, half size)
    """
    if factor != 1.0:
        img = cv2.resize(img, (0,0), fx=factor, fy=factor) 

    cv2.imshow(name,img)
    while(1):
        k = cv2.waitKey(0)
        if k==27:    # Esc key to stop
            break
    cv2.destroyAllWindows()

In [3]:
def read_image(input_path):
    img = cv2.imread(input_path,0)
    return img

In [4]:
def crop_img(img):
    mask_inv = img
    ver_sum = np.sum(mask_inv,axis=1)
    v_start = 0
    v_end = 0
    for i in range(len(ver_sum)):
        if(ver_sum[i] > 0 and v_start ==0):
            v_start = i
        if(ver_sum[i] == 0 and v_start != 0):
            v_end = i
            break
    if(v_end == 0):
        v_end = len(ver_sum) - 1
    
    hor_sum = np.sum(mask_inv,axis=0)
    h_start = 0
    h_end = 0
    for i in range(len(hor_sum)):
        if(hor_sum[i] > 0 and h_start ==0):
            h_start = i
        if(hor_sum[i] == 0 and h_start != 0):
            h_end = i
            break
    if(h_end == 0):
        h_end = len(hor_sum) - 1

    return img[v_start:v_end,h_start:h_end]

## Pre-Processing

In [5]:
def preprocess_img(img,show_steps=1,show_size=0.2):
    img = cv2.GaussianBlur(img,(11,11),0)
    thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img[img >= thresh[0]] = 255
    img[img <= thresh[0]] = 0
    img = cv2.bitwise_not(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    if show_steps == 1:        
        show(img,show_size,"Preprocessed Image")
    return img

In [6]:
def remove_top(img,show_steps=1,show_size=0.2):
    trsh = 0.1
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.dilate(img,kernel,iterations = 8)
    proj = np.sum(erosion,axis=1).astype(int)
    if show_steps == 1:        
        show(erosion,show_size,"Eroded")
    max_line = np.amax(proj)

    lines = []
    lines.append(300)
    for i in range(img.shape[0]):
        if(proj[i] >= trsh*max_line) :
            if(len(lines) > 0 ):
                if(lines[-1] + img.shape[0]/10 > i ):
                    continue
            if( (i>=500 and i <= 900) or (i>=2600 and i <= 2900)):
                lines.append(i)
    if show_steps == 1:        
        show(img[lines[1]:lines[2],:],show_size,"Cropped Image")
    
    return img[lines[1]:lines[2],:]


In [7]:
def sort_lists(list1,list2):
    list1 = np.asarray(list1)
    sorter = np.argsort(list1)
    list3 = []
    for i in range(len(sorter)):
        list3.append(list2[sorter[i]])
    return list3

In [8]:
def merge_boxes(boxes,max_h):
    i = 0
    while i < len(boxes)-1:
        j = i+1
        while j < len(boxes):
            x1,y1,w1,h1 = (boxes[i])[0],(boxes[i])[1],(boxes[i])[2],(boxes[i])[3]
            x2,y2,w2,h2 = (boxes[j])[0],(boxes[j])[1],(boxes[j])[2],(boxes[j])[3]
            if x2 < x1+w1:
                boxes[i] = [x1,min(y1,y2),x2+w2-x1,max(y1+h1,y2+h2)-min(y1,y2)]
                del boxes[j]
            else:
                break
        i += 1
    return boxes

In [9]:
def get_sentences(img,show_steps=1,show_size=0.2):
    
    original = img.copy()
    blurred = cv2.GaussianBlur(img, (1, 1), 0)
    canny = cv2.Canny(blurred, 50, 255, 1)
    kernel = np.ones((1,7),np.uint8)
    dilate = cv2.dilate(canny, kernel, iterations=40)
    
    if show_steps == 1:        
        show(dilate,show_size,"Dialted for Sentence Extraction")
    
    # Find contours
    cnts,_ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate thorugh contours and filter for ROI
    image_number = 0
    order_list = []
    images = []
    area = 0 
    avg_height = 0
    for c in cnts:
        area += cv2.contourArea(c)
    area /= (len(cnts))
    for c in cnts:
        if cv2.contourArea(c) > area/1.5 :
            x,y,w,h = cv2.boundingRect(c)
            if h > img.shape[0]/5:
                continue
            cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
            ROI = original[y:y+h, x:x+w]
            hor_sum = np.sum(ROI,axis=1)
            if cv2.contourArea(c) > area*2.5:
                images.append(ROI[0:int(ROI.shape[0]/2),:])
                order_list.append(y)
                images.append(ROI[int(ROI.shape[0]/2):,:])
                order_list.append(y+1)     
            else:
                images.append(ROI)
                order_list.append(y)
            avg_height += h
    avg_height /= len(order_list)
    sentences = sort_lists(order_list,images)
    return sentences,avg_height

In [10]:
def get_horizontal_merge(sentence,show_steps=1,show_size=0.2):

    original = np.copy(sentence)
    # Find contours
    cnts,_ = cv2.findContours(sentence, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Iterate thorugh contours and filter for ROI
    order_list = []
    boxes = []
    images = []
    area = 0 
    for c in cnts:
        area += cv2.contourArea(c)
    area /= len(cnts)
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        if cv2.contourArea(c) > area/5 :
            boxes.append([x,y,w,h])
            order_list.append(x)
        else:
            original[y:y+h,x:x+w] = 0
    boxes = sort_lists(order_list,boxes)
    boxes = merge_boxes(boxes,original.shape[0])
    for box in boxes:
        x,y,w,h = box[0],box[1],box[2],box[3]                    
        ROI = np.zeros((original.shape[0],w))
        if int(original.shape[0]/2-h/2) > 0 :
            ROI[int(original.shape[0]/2-h/2):int(original.shape[0]/2-h/2)+h,:] = original[y:y+h,x:x+w]
        else:           
            ROI[0:h,:] = original[y:y+h,x:x+w]
        images.append(ROI)
    
    hori_merged = np.zeros((original.shape[0],original.shape[1]))
    current = 0
    for image in images:
        hori_merged[:,current:current+image.shape[1]] += image
        current = current+image.shape[1]
    hori_merged = crop_img(hori_merged)
    if show_steps == 1:        
        show(hori_merged,show_size*5,"Sentence Horizontally Merge")

    return hori_merged

In [11]:
def rearrange_image(image,show_steps=1,show_size=0.2):
    
    copy = np.zeros((image.shape[0], image.shape[1]))
    sentences,avg_height = get_sentences(image,show_steps,show_size)
    contours, hierarchy = cv2.findContours(image.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    currentY = 150
    for i, sentence in enumerate(sentences):
        sentence = get_horizontal_merge(sentence,show_steps,show_size)
        copy[currentY:currentY+sentence.shape[0],0:sentence.shape[1]] += sentence
        currentY += int(avg_height/2)
    copy = crop_img(copy)
    if show_steps == 1:        
        show(copy,show_size*2,"Vertical Merge")
    return copy

In [12]:
def divide_image(image,show_steps=1,show_size=0.2):
    factor = 3
    height, width = image.shape
    img_arr = []
    w_3 =int(width/3)
    h_3= int(height/3)
    #first row
    img_arr.append(image[0:h_3 , 0:w_3])
    img_arr.append(image[0:h_3 , w_3:2*w_3])
    img_arr.append(image[0:h_3 , 2*w_3:3*w_3])
    
    #second row
    img_arr.append(image[h_3:2*h_3 , 0:w_3])
    img_arr.append(image[h_3:2*h_3 , w_3:2*w_3])
    img_arr.append(image[h_3:2*h_3 , 2*w_3:3*w_3])
    
    #third row
    img_arr.append(image[2*h_3:3*h_3 , 0:w_3])
    img_arr.append(image[2*h_3:3*h_3 , w_3:2*w_3])
    img_arr.append(image[2*h_3:3*h_3 , 2*w_3:3*w_3])
    if show_steps == 1:
        for img in img_arr:
            show(img,show_size*2,"Texture Block")
    return img_arr

In [13]:
def get_pixel(img, center, x, y): 
      
    new_value = 0
      
    try: 
        # If local neighbourhood pixel  
        # value is greater than or equal 
        # to center pixel values then  
        # set it to 1 
        if img[x][y] >= center: 
            new_value = 1
              
    except: 
        # Exception is required when  
        # neighbourhood value of a center 
        # pixel value is null i.e. values 
        # present at boundaries. 
        pass
      
    return new_value 
   
# Function for calculating LBP 
def lbp_calculated_pixel(img, x, y): 
   
    center = img[x][y] 
   
    val_ar = [] 
      
    # top_left 
    val_ar.append(get_pixel(img, center, x-1, y-1)) 
      
    # top 
    val_ar.append(get_pixel(img, center, x-1, y)) 
      
    # top_right 
    val_ar.append(get_pixel(img, center, x-1, y + 1)) 
      
    # right 
    val_ar.append(get_pixel(img, center, x, y + 1)) 
      
    # bottom_right 
    val_ar.append(get_pixel(img, center, x + 1, y + 1)) 
      
    # bottom 
    val_ar.append(get_pixel(img, center, x + 1, y)) 
      
    # bottom_left 
    val_ar.append(get_pixel(img, center, x + 1, y-1)) 
      
    # left 
    val_ar.append(get_pixel(img, center, x, y-1)) 
       
    # Now, we need to convert binary 
    # values to decimal 
    power_val = [1, 2, 4, 8, 16, 32, 64, 128] 
   
    val = 0
      
    for i in range(len(val_ar)): 
        val += val_ar[i] * power_val[i] 
          
    return val 
   

def get_LBP(sentence,show_steps=1,show_size=0.2):

    height, width = sentence.shape 


    # Create a numpy array as  
    # the same height and width  
    # of RGB image 
    img_lbp = np.zeros((height, width), np.uint8) 

    for i in range(0, height): 
        for j in range(0, width): 
            img_lbp[i, j] = lbp_calculated_pixel(sentence, i, j) 
    return img_lbp
#img = get_LBP(sentences[0])
#show(img)

# Main

In [14]:
input_path = 'F:/Tech/CUFE_CHS/Fall_2020/Pattern/Project/Data_Set/'#a02-000.png'
show_steps = 0
show_size = 0.2

images = []
for file in os.listdir(input_path):
    images.append(file)
images.sort()
counter = 0
for input_img in images:
    try:
        img = read_image(input_path+input_img)
    # img = read_image(input_path)#+input_img)
        img = preprocess_img(img,show_steps,show_size)
        img = remove_top(img,show_steps,show_size)
        large_texture_block = rearrange_image(img,show_steps,show_size)
        small_texture_blocks = divide_image(large_texture_block,show_steps,show_size)
    except:
        print("Error at : " , input_img)
        counter += 1
print(counter)

Error at :  a01-011x.png
Error at :  a01-058.png
Error at :  b04-081.png
Error at :  b04-089.png
Error at :  b04-103.png
Error at :  b06-027.png
Error at :  c03-016c.png
Error at :  c04-044.png
Error at :  c06-103.png
Error at :  e02-021.png
Error at :  e02-082.png
Error at :  e06-026.png
Error at :  f01-081.png
Error at :  formsA-D.tgz
Error at :  formsE-H.tgz
Error at :  formsI-Z.tgz
Error at :  g03-004.png
Error at :  h07-075a.png
Error at :  h07-078a.png
Error at :  h07-080a.png
Error at :  h07-084.png
Error at :  k02-029.png
Error at :  k04-126.png
Error at :  l03-000.png
Error at :  l07-135.png
Error at :  m01-079.png
Error at :  m04-061.png
Error at :  n01-052.png
Error at :  n02-028.png
Error at :  n02-049.png
Error at :  n04-149.png
31


# Extra

In [ ]:
def training_data(folder_name):
    input_path = "/media/madmax/MadMax_D/CUFE/Fall_2021/CMPN450_PatternRecognition_and_NeuralNetworks/Project document/Writer-Recognition/data/" + folder_name + "/"
    images = []
    labeles=[]
    # iterate over all 3 writers and add their feature vectors
    for i in range(1, 4):
        newpath = input_path + str(i) + "/"
        for file in os.listdir(newpath):
            images.append(cv2.imread(newpath + file))
            labeles.append(i)

    return images, labeles

In [ ]:
test_cases_count=10
images = []
lables=[]
for test_cases in range (1,test_cases_count + 1):
    folder_name = str(test_cases)
    while len(folder_name)<2:
        folder_name = "0" + folder_name
    images_temp,lables_temp = training_data(folder_name)
    images.append(images_temp)
    lables.append(lables_temp)